In [14]:
from selenium.webdriver.common.by import By
import undetected_chromedriver.v2 as uc
from pyvirtualdisplay import Display
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from transformers import pipeline
from neo4j import GraphDatabase
import torch

def extract_text_by_class(class_name):
    """
    Extract text from an element with the specified class name
    """
    global wd
    try:
        content = wd.find_element(By.CLASS_NAME, class_name)
        return content.text
    except:
        return ""


def extract_links_by_xpath(xpath):
    global wd
    links = set()
    try:
        a_elems = wd.find_elements(By.XPATH, xpath)
        for elem in a_elems:
            link = elem.get_attribute("href")
            if link == "javascript:void(0)":
                continue
            # Resimlere ve çeşitli dosyaları filtreleme işlemi
            if (
                link.endswith(".png")
                or link.endswith(".json")
                or link.endswith(".txt")
                or link.endswith(".svg")
                or link.endswith(".ipynb")
                or link.endswith(".jpg")
                or link.endswith(".pdf")
                or link.endswith(".mp4")
                or "mailto" in link
                or len(link) > 300
            ):
                continue
            # Remove anchors
            link = link.split("#")[0]
            # Remove parameters
            link = link.split("?")[0]
            # Remove trailing forward slash
            link = link.rstrip("/")
            links.add(link)
        return list(links)
    except:
        return []

In [15]:
tokenizer = AutoTokenizer.from_pretrained("yanekyuk/bert-uncased-keyword-extractor")
model = AutoModelForTokenClassification.from_pretrained(
    "yanekyuk/bert-uncased-keyword-extractor"
)

nlp = pipeline("ner", model=model, tokenizer=tokenizer)


def extract_keywords(text):
    """
    Extract keywords and construct them back from tokens
    """
    result = list()
    keyword = ""
    for token in nlp(text):
        if token["entity"] == "I-KEY":
            keyword += (
                token["word"][2:]
                if token["word"].startswith("##")
                else f" {token['word']}"
            )
        else:
            if keyword:
                result.append(keyword)
            keyword = token["word"]
    # Add the last keyword
    result.append(keyword)
    return list(set(result))

In [16]:
extract_keywords(
    """
Broadcom agreed to acquire cloud computing company VMware in a $61 billion (€57bn) cash-and stock deal.
"""
)

['cloud computing', 'vmware', 'broadcom']

In [17]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


def generate_embeddings(text):
    embeddings = model.encode(text)
    return [float(x) for x in embeddings.tolist()]

In [18]:
generate_embeddings(
    """
Web APIs are a huge opportunity to access and integrate data from any sources with your graph. Most of them provide the data in JSON format.

The Load JSON procedures retrieve data from URLs or maps and turn it into map value(s) for Cypher to consume. Cypher has support for deconstructing nested documents with dot syntax, slices, UNWIND etc. so it is easy to turn nested data into graphs.

Sources with multiple JSON objects (JSONL,JSON Lines) in a stream, like the streaming Twitter format or the Yelp Kaggle dataset, are also supported,
"""
)

[-0.06911398470401764,
 0.05224500596523285,
 -0.00720181642100215,
 -0.015374762937426567,
 0.0039060309063643217,
 -0.050424594432115555,
 -0.10675833374261856,
 0.0622570626437664,
 0.0032547342125326395,
 0.013045622035861015,
 0.0074321553111076355,
 0.002438637660816312,
 0.0732424333691597,
 0.06017127260565758,
 0.13454288244247437,
 0.03666399046778679,
 -0.007088588550686836,
 0.030180800706148148,
 0.026259901002049446,
 -0.1478315144777298,
 0.011475404724478722,
 0.0016846579965204,
 -0.04652785137295723,
 -0.037655558437108994,
 0.04880434274673462,
 0.08559447526931763,
 -0.03026597388088703,
 0.052127715200185776,
 0.04567538946866989,
 0.06373853236436844,
 -0.02977648377418518,
 -0.010671388357877731,
 -0.05804383009672165,
 0.05813640356063843,
 -0.09820526838302612,
 0.009667870588600636,
 0.03530017286539078,
 0.00953008234500885,
 0.013390502892434597,
 0.05266077071428299,
 0.042114634066820145,
 0.012197659350931644,
 -0.047448016703128815,
 0.018271835520863533

In [19]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from sentence_transformers import SentenceTransformer

def extract_text_by_class(class_name):
    """
    Extract text from an element with the specified class name
    """
    global wd
    try:
        content = wd.find_element(By.CLASS_NAME, class_name)
        return content.text
    except:
        return ""

def extract_links_by_xpath(xpath):
    global wd
    links = set()
    try:
        a_elems = wd.find_elements(By.XPATH, xpath)
        for elem in a_elems:
            link = elem.get_attribute("href")
            if link == "javascript:void(0)":
                continue
            
            if (
                link.endswith(".png")
                or link.endswith(".json")
                or link.endswith(".txt")
                or link.endswith(".svg")
                or link.endswith(".ipynb")
                or link.endswith(".jpg")
                or link.endswith(".pdf")
                or link.endswith(".mp4")
                or "mailto" in link
                or len(link) > 300
            ):
                continue
            
            link = link.split("#")[0]
            
            link = link.split("?")[0]
            
            link = link.rstrip("/")
            links.add(link)
        return list(links)
    except:
        return []


model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


def generate_embeddings(text):
    embeddings = model.encode(text)
    return [float(x) for x in embeddings.tolist()]


options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
wd = webdriver.Chrome( options=options)




entry_url = "https://neo4j.com/docs"
data = dict()
visit_list = [entry_url]
already_visited = []
visited_links_count = 0  # Sayaç

while visit_list and visited_links_count < 20:  # 100'e ulaşana kadar devam et
    
    current_url = visit_list.pop()
    if current_url in already_visited:
        continue
    print(current_url)
    try:
        wd.get(current_url)
    except:
        print(f"Couldn't open {current_url}")
        already_visited.append(current_url)
        continue
    
    try:
        actual_url = wd.current_url.rstrip("/").split("#")[0].split("?")[0]
        if actual_url != current_url:
            
            data[current_url] = {
                "links": [],
                "text": None,
                "embeddings": [],
                "keywords": [],
                "redirects": [actual_url],
            }
            already_visited.append(current_url)
            
            current_url = actual_url
    except:
        pass
    
    text = extract_text_by_class("content")
    
    if not text:
        text = extract_text_by_class("article")
   
    if not text:
        text = extract_text_by_class("page")
    if not text:
        text = extract_text_by_class("single-user-story")
    
    try:
        if "Sorry, page not found" in wd.find_element(By.TAG_NAME, "body").text:
            text = "404"
    except:
        pass

    
    if text:
        embeddings = generate_embeddings(text)
        keywords = extract_keywords(text)
    else:
        embeddings = []
        keywords = []

 
    links = extract_links_by_xpath("//div[@class='content']//a[@href]")
    
    if not links:
        links = extract_links_by_xpath("//article[@class='article']//a[@href]")
    if not links:
        links = extract_links_by_xpath("//article//a[@href]")

   
    data[current_url] = {
        "links": [l for l in links if l != current_url],
        "text": text,
        "embeddings": embeddings,
        "keywords": keywords,
        "redirects": [],
    }
    
    already_visited.append(current_url)
    visited_links_count += 1  
    visit_list.extend(
        [
            l
            for l in list(links)
            if ("neo4j.com" in l)
            and (not l in already_visited)
            and (not "community.neo4j.com" in l)
            and (not "sandbox.neo4j.com" in l)
        ]
    )


wd.quit()

https://neo4j.com/docs
https://neo4j.com/docs/create-applications
https://neo4j.com/docs/graphql-manual/current
https://neo4j.com
https://neo4j.com/docs/graphql/current/getting-started
https://neo4j.com/docs/graphql/5
https://neo4j.com/docs/graphql/5/directives/custom-logic
https://neo4j.com/docs/graphql/5/directives/custom-directives
https://neo4j.com/docs/graphql/5/queries-aggregations
https://neo4j.com/docs/graphql/5/queries-aggregations/sorting
https://neo4j.com/docs/graphql/5/queries-aggregations/filtering
https://neo4j.com/docs/graphql/5/types/scalar
https://neo4j.com/docs/graphql/5/types
https://neo4j.com/docs/graphql/5/types/temporal
https://neo4j.com/developer/cypher/dates-datetimes-durations
https://neo4j.com/docs/cypher-manual/current/functions/temporal
https://neo4j.com/docs/cypher-manual/5/functions/string
https://neo4j.com/docs/cypher-manual/5/functions/mathematical-trigonometric
https://neo4j.com/docs/cypher-manual/5/syntax/operators
https://neo4j.com/docs/cypher-manual/

In [20]:
# Neo4j bağlantı ayarları
uri = "neo4j+s://61a02345.databases.neo4j.io"
user = "neo4j"
password = "VvjpT-s9bsygPEVFe3ekLYS9NHtwFRPvXloYYOveQME"

try:
    driver = GraphDatabase.driver(uri, auth=(user, password))
    with driver.session() as session:
        result = session.run("RETURN 1")
        for record in result:
            print(record)
except Exception as e:
    print("Error:", e)

<Record 1=1>


In [21]:
# yalnızca GDM ile çalıştır

from neo4j import GraphDatabase
import json

j = json.dumps(data)

f = open("neo4j_docs.json", "w")


f.write(j)


f.close()


# Neo4j bağlantı ayarları
uri = "neo4j+s://61a02345.databases.neo4j.io"
user = "neo4j"
password = "VvjpT-s9bsygPEVFe3ekLYS9NHtwFRPvXloYYOveQME"
driver = GraphDatabase.driver(uri, auth=(user, password))


with driver.session() as session:
   
    j = json.dumps(data)
    with open("neo4j_docs.json", "w") as f:
        f.write(j)

    
    session.run("""
        CREATE CONSTRAINT IF NOT EXISTS FOR (p:Page) REQUIRE p.url IS UNIQUE;
    """)
    
    session.run("""
        CREATE CONSTRAINT IF NOT EXISTS FOR (k:Keyword) REQUIRE k.name IS UNIQUE;
    """)

driver.close()

In [22]:
# yalnızca GDM ile çalıştır

import json

from neo4j import GraphDatabase

uri = "neo4j+s://61a02345.databases.neo4j.io"
user = "neo4j"
password = "VvjpT-s9bsygPEVFe3ekLYS9NHtwFRPvXloYYOveQME"

driver = GraphDatabase.driver(uri, auth=(user, password))

import_query = """
UNWIND $data AS row
MERGE (p:Page {url:row.url})
SET p.embedding = row.embedding,
    p.has_text = row.has_text,
    p.is_404 = row.is_404
FOREACH (l in row.links    | MERGE (p1:Page {url:l}) MERGE (p)-[:LINKS_TO]->(p1))
FOREACH (k in row.keywords | MERGE (k1:Keyword {name:k}) MERGE (p)-[:HAS_KEYWORD]->(k1))
FOREACH (r in row.redirects| MERGE (r1:Page {url:r}) MERGE (p)-[:REDIRECTS]->(r1))
"""

x = 1
params = []

for key in data:
    params.append(
        {
            "url": key,
            "embedding": data[key]["embeddings"],
            "keywords": data[key]["keywords"],
            "links": data[key]["links"],
            "has_text": True if data[key]["text"] else False,
            "is_404": True if data[key]["text"] == "404" else False,
            "redirects": data[key]["redirects"],
        }
    )

    
    if len(params) == 500:
        with driver.session() as session:
            session.run(import_query, {"data": params})
            params = []
            # Logging
            print(f"Importing {x} batch")
            x += 1


with driver.session() as session:
    session.run(import_query, {"data": params})

driver.close()

In [23]:
from neo4j import GraphDatabase

# Neo4j bağlantı ayarları
uri = "neo4j+s://61a02345.databases.neo4j.io"
user = "neo4j"
password = "VvjpT-s9bsygPEVFe3ekLYS9NHtwFRPvXloYYOveQME"

driver = GraphDatabase.driver(uri, auth=(user, password))

query = """
CALL apoc.meta.stats()
YIELD labels, relTypesCount
"""

with driver.session() as session:
    result = session.run(query)
    for record in result:
        print(record["labels"], record["relTypesCount"])

driver.close()

{'Keyword': 47, 'Page': 153} {'HAS_KEYWORD': 67, 'LINKS_TO': 209, 'REDIRECTS': 4}


In [24]:
from neo4j import GraphDatabase

# Neo4j bağlantı ayarları
uri = "neo4j+s://61a02345.databases.neo4j.io"
user = "neo4j"
password = "VvjpT-s9bsygPEVFe3ekLYS9NHtwFRPvXloYYOveQME"

driver = GraphDatabase.driver(uri, auth=(user, password))

query = """
MATCH (p:Page)
RETURN p.has_text AS has_text,
       count(*) AS count
"""

with driver.session() as session:
    result = session.run(query)
    for record in result:
        print(record["has_text"], record["count"])

driver.close()

True 20
False 4
None 129


In [25]:
# Neo4j bağlantı ayarları
uri = "neo4j+s://61a02345.databases.neo4j.io"
user = "neo4j"
password = "VvjpT-s9bsygPEVFe3ekLYS9NHtwFRPvXloYYOveQME"

driver = GraphDatabase.driver(uri, auth=(user, password))

query = """
MATCH (p:Page)
WHERE p.has_text IS NULL
WITH p, [(p)<-[:LINKS_TO|REDIRECTS]-() | 1] AS links
RETURN p.url AS page, size(links) AS links_count
ORDER BY links_count DESC
LIMIT 5
"""

with driver.session() as session:
    result = session.run(query)
    for record in result:
        print(record["page"], record["links_count"])

driver.close()

https://neo4j.registration.goldcast.io/events/03805ea9-fe3a-4cac-8c15-aa622666531a 6
https://neo4j.com/docs/graphql/5/queries-aggregations/pagination 3
https://neo4j.com/docs/license 3
https://neo4j.com/docs/apoc/current 3
https://graphacademy.neo4j.com/courses/graphql-basics 3


In [26]:
# Neo4j bağlantı ayarları
uri = "neo4j+s://61a02345.databases.neo4j.io"
user = "neo4j"
password = "VvjpT-s9bsygPEVFe3ekLYS9NHtwFRPvXloYYOveQME"

driver = GraphDatabase.driver(uri, auth=(user, password))

query = """
MATCH (p:Page)-[:HAS_KEYWORD]->(k:Keyword)
RETURN p.url AS page,
       COLLECT(k.name) AS keywords
"""

with driver.session() as session:
    result = session.run(query)
    for record in result:
        print(record["page"], record["keywords"])

driver.close()

https://neo4j.com/docs ['database', 'aura', 'query language', 'creative commons']
https://neo4j.com/docs/graphql/5/types/scalar ['database', 'graphql', 'neo4j', 'neo4']
https://neo4j.com/docs/create-applications ['python', 'javascript', 'neo', 'java', 'graphql', '. net']
https://neo4j.com/docs/graphql/current ['javascript', 'graphql', 'graph', 'open source', 'apollo', 'graphq']
https://neo4j.com/docs/graphql/5/getting-started ['javascript', 'graphql', 'graph', 'graphq', 'graphacademy', 'apollo server', 'command line']
https://neo4j.com/docs/graphql/5 ['javascript', 'graphql', 'graph', 'open source', 'apollo', 'graphq']
https://neo4j.com/docs/cypher-manual/5/values-and-types/maps ['java']
https://neo4j.com/docs/graphql/5/directives/custom-logic ['graphql']
https://neo4j.com ['whiteboard', 'organizationslogy']
https://neo4j.com/docs/graphql/5/directives/custom-directives ['graphql - tools', 'graphql toolserer', 'directive', 'neo4j graphql']
https://neo4j.com/docs/graphql/5/queries-aggreg